# UdaciMed | Notebook 1: Baseline Analysis

Welcome to UdaciMed! You are a Machine Learning Engineer tasked with optimizing our chest X-ray pneumonia detection model for production deployment.

- **The challenge**: Before our new model can be approved for production, we must verify that it meets our strict performance SLAs. Deploying an unoptimized model could lead to high operational costs and poor performance across our diverse hardware fleet—from shared T4 GPU cloud instances to portable clinic devices.

- **Your mission**: Optimize the model to pass _UdaciMed's Universal Performance Standard_, a strict production service level agreement (SLA) that must be met using the universally compatible ONNX format on our standardized target device.

### **Optimization goals**

Your goal is to ensure the production model meets these production targets on our standardized development machine:

- **< 0.4 GFLOPs per sample**: Floating Point Operations determine computational cost - reducing FLOPs is the most critical step toward broad-platform efficiency.
- **< 100MB peak memory footprint**: Total memory consumption (parameters + activations + workspace) during inference - essential for running the model on memory-constrained edge devices and for enabling cost-effective multi-model environments in the cloud.
- **< 3ms latency**: This ensures a real-time user experience. We will measure both *amortized latency* (average time per sample in a large batch) and *true latency* (time for a single-image inference), as both are important for different use cases.
- **> 2000 samples/second throughput**: This specific target is for our high-performance hardware, like the reference T4 GPU. Meeting this goal proves the model is cost-effective and scalable for high-volume, server-side screening workflows.
- **> 98% sensitivity***: This is a non-negotiable clinical safety requirement. We must ensure that a threshold percentage of all true pneumonia cases are correctly identified. All optimizations must be validated against this metric.

#### **A note on our standardized target device**

All performance targets in this project must be met on our official _"standardized target device."_ This is an NVIDIA T4 GPU, a common and versatile datacenter GPU that represents a typical cloud deployment environment.

By using a single, consistent hardware profile (NVIDIA T4 with 16GB VRAM, CUDA 12.4) for all our performance SLAs, we can:

- *Ensure Reproducible Results*: Anyone on the team can validate performance and get consistent measurements.

- *Create a Reliable Benchmark*: It provides a stable baseline to measure the impact of every optimization we make.

If a model can meet our strict, universal performance standards on this reference hardware, we are confident it will perform well across our entire fleet of production devices.

---

Through this notebook, you will build the foundation for our optimization strategy by:

1.  **Establishing baseline performance** with comprehensive profiling.
2.  **Analyzing the primary bottlenecks**, distinguishing between compute, parameter memory, and activation memory.
3.  **Identifying optimization opportunities** in both the model architecture and the deployment configuration.

**Let's set up an optimization vision for UdaciMed's next-generation diagnostic platform!**


> **_\*Understanding medical AI requirements_**:
> 
> In medical AI, sensitivity (recall) is often more critical than overall accuracy. Missing a pneumonia case (false negative) can be life-threatening, while a false positive "only" leads to additional human review. This is why we prioritize sensitivity as our safety constraint.

## Step 1: Set up the environment
The first step is to import all libraries and internal functionalities (from `utils`). 

Additionally, we set `pytorch` to use CUDA GPU if available (not only for faster execution, but also for benchmarking, as this will be our final deployment target!) and we include deterministic mode for reproducible benchmarking.

In [ ]:
# Make sure that libraries are dynamically re-loaded if changed
%load_ext autoreload
%autoreload 2

In [ ]:
# Import core libraries
import torch
import torch.nn as nn
import inspect
import numpy as np
import os
import pickle
from pprint import pprint
import random
from torchsummary import summary
import tqdm
import warnings
warnings.filterwarnings('ignore')

# Import project utilities

# Ensure project root is on sys.path so 'utils' imports work when running from the notebooks folder
import sys, os
if os.path.basename(os.getcwd()) == 'notebooks' and os.path.exists('..'):
    sys.path.append('..')
from utils.data_loader import (
    load_pneumoniamnist, 
    get_dataset_info, 
    explore_dataset_splits,
    visualize_sample_images,
    get_sample_batch
)
from utils.model import (
    create_baseline_model,
    get_model_info,
    count_parameters_by_type,
    train_baseline_model,
    plot_training_history
)
from utils.evaluation import (
    evaluate_with_multiple_thresholds
)
from utils.profiling import (
    PerformanceProfiler,
    get_gpu_info,
    check_environment,
    measure_time
)
from utils.visualization import (
    plot_dataset_distribution,
    plot_performance_profile,
    plot_operation_breakdown,
    plot_batch_size_comparison,
)

# Check environment and GPU capabilities
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if torch.cuda.is_available():
    gpu_info = get_gpu_info()
    print(f"GPU: {gpu_info.get('name', 'Unknown')}")
    print(f"GPU Memory: {gpu_info.get('memory_total_gb', 0):.1f} GB")
    print(f"Tensor Core Support: {gpu_info.get('tensor_core_support', False)}")
else:
    print("WARNING: CUDA not available - profiling will be limited")

In [ ]:
# Set random seed for reproducibility across optimization experiments
def set_deterministic_mode(seed=42):
    """
    Enable deterministic mode for consistent benchmarking.
    Critical for fair comparison between different techniques.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  # Disable for consistent timing
    os.environ["PYTHONHASHSEED"] = str(seed)

set_deterministic_mode(42)
print("Deterministic mode enabled for reproducible benchmarking")

## Step 2: Load and analyze the dataset
Now, we can get started with our baseline model set-up by loading the data and understanding its characteristics. 

For this project, we use the PneumoniaMNIST dataset from [MedMNIST](https://medmnist.com/). PneumoniaMNIST provides a standardized, validated dataset for pneumonia detection research. Its 64x64 resolution balances clinical detail with computational efficiency, making it ideal for optimization studies while maintaining diagnostic relevance.

In [ ]:
# Get dataset information
dataset_info = get_dataset_info(use_binary=True)
print("PneumoniaMNIST Dataset Information:")
for key, value in dataset_info.items():
    print(f"   {key.replace('_', ' ').title()}: {value}")

In [ ]:
# Define configuration for baseline analysis
CONFIG = {
    'image_size': 64,  # Balanced for for memory usage and model accuracy
    'num_classes': 2,  # Binary classification: normal vs pneumonia
    'batch_size': 32,  # Balanced for memory usage and training stability
    'subset_size': 800,  # Use a subset for faster execution during automated runs
}

# Load the dataset with optimized settings
print("Loading PneumoniaMNIST dataset...")

with measure_time("Dataset loading"):
    train_loader = load_pneumoniamnist(
        split="train", download=True, 
        size=CONFIG['image_size'], batch_size=CONFIG['batch_size'], 
        subset_size=int(CONFIG['subset_size'] * 0.7) if CONFIG['subset_size'] is not None else None
    )
    
    val_loader = load_pneumoniamnist(
        split="val", download=False, 
        size=CONFIG['image_size'], batch_size=CONFIG['batch_size'], 
        subset_size=int(CONFIG['subset_size'] * 0.15) if CONFIG['subset_size'] is not None else None
    )
    
    test_loader = load_pneumoniamnist(
        split="test", download=False, 
        size=CONFIG['image_size'], batch_size=CONFIG['batch_size'], 
        subset_size=int(CONFIG['subset_size'] * 0.15) if CONFIG['subset_size'] is not None else None
    )

print(f"Dataset loaded: {CONFIG['image_size']}x{CONFIG['image_size']} images, batch_size={CONFIG['batch_size']}")

In [ ]:
# Analyze dataset distribution for class imbalance considerations
print("Analyzing dataset distribution...")
dataset_splits = explore_dataset_splits(train_loader, val_loader, test_loader)
print(f"\nDataset Summary: {dataset_splits}")

# Visualize dataset distribution
plot_dataset_distribution(dataset_splits)

> **_The impact of class imbalance_**
> 
> Medical datasets often have class imbalance. This affects optimization because:
> 
> - Models may focus compute on majority class features
> - Batch composition affects memory usage patterns
> - Some optimization techniques (like pruning) may disproportionately affect minority class performance

In [ ]:
# Visualize sample images to understand data characteristics
print("Sample chest X-ray images:")
visualize_sample_images(train_loader, num_samples=4)

## Step 3: Create and analyze the baseline model
We will use [ResNet-18](https://docs.pytorch.org/vision/main/models/generated/torchvision.models.resnet18.html) as our baseline - a popular choice for medical imaging that balances accuracy and efficiency.

The original model structure is architected for ImageNet (1000 classes), so we modify the model with a custom head to support our new classification task.

In [ ]:
# Create the baseline ResNet-18 model
print("Creating ResNet-18 baseline model...")

baseline_model = create_baseline_model(
    num_classes=CONFIG['num_classes'], 
    input_size=CONFIG['image_size'], 
    pretrained=False  # Training from scratch for fair optimization comparison
)
baseline_model = baseline_model.to(device)

print(f"Baseline model created and deployed to {device}")
print(f"   Architecture: {baseline_model.architecture_name}")

In [ ]:
# Get model information
model_info = get_model_info(baseline_model)

print(f"\nModel Information:")
print(f"   Architecture: {model_info['architecture']}")
print(f"   Total Parameters: {model_info['total_parameters']:,}")
print(f"   Model Size: {model_info['model_size_mb']:.1f} MB")
print(f"   Input Size: {model_info['input_size']}x{model_info['input_size']}")

# Analyze layer composition 
layer_breakdown = model_info['layer_breakdown']
print(f"\nLayer Composition:")
print(f"   Convolution Layers: {layer_breakdown['conv_layers']['count']} ({layer_breakdown['conv_layers']['total_params']:,} params)")
print(f"   Linear Layers: {layer_breakdown['linear_layers']['count']} ({layer_breakdown['linear_layers']['total_params']:,} params)")
print(f"   Normalization Layers: {layer_breakdown['norm_layers']['count']}")
print(f"   Activation Types: {', '.join(layer_breakdown['activation_layers']['types'])}")

# Get parameter distribution
if 'parameter_distribution' in layer_breakdown:
    param_dist = layer_breakdown['parameter_distribution']
    print(f"\nParameter Distribution:")
    print(f"   Convolution: {param_dist['conv_percentage']:.2f}%")
    print(f"   Linear: {param_dist['linear_percentage']:.2f}%")

In [ ]:
# Display the model architecture
summary(baseline_model, input_size=(3, CONFIG["image_size"], CONFIG["image_size"]))

> **_Brainstorming time: Optimizations by layer type_**
> 
> Try to remember from the course, which architectural optimizations most benefit each layer type? Looking at the layer composition, our model is convolution-heavy - this is your starting point!

## Step 4: Train and evaluate baseline model
Now, we define the baseline model and evaluate its performance on key accuracy metrics - all future optimizations should maintain a similar clinical performance standard.

Establishing robust baseline metrics is crucial for medical AI. Any optimization must preserve clinical safety while improving computational efficiency.

In [ ]:
# Training configuration
TRAINING_CONFIG = {
    'num_epochs': 3,            # Reduced for faster baseline execution
    'learning_rate': 0.0003,    # Conservative rate for stable training
    'lr_step_size': 3,          # Learning rate decay schedule
    'weight_decay': 1e-4,       # Regularization for generalization
    'patience': 1               # Early stopping for efficiency
}

# Train the model
baseline_model, training_history = train_baseline_model(
    baseline_model, train_loader, val_loader, device, TRAINING_CONFIG
)

# Plot training curves with analysis
plot_training_history(training_history)

> **Training insights: What does the training history tell us?**
> 
> The initial low validation accuracy is due to extreme overfitting due to the small dataset size in comparison to the model's parameter size. The presence of early plateaus and fast convergence also highlight that the architecture has high representational power - we can likely apply aggressive optimization without accuracy degradation.

In [ ]:
# Evaluate model performance with multiple thresholds 
print(f"Running eval benchmark on {dataset_splits['test']['total']} test samples ...")
eval_results = evaluate_with_multiple_thresholds(baseline_model, test_loader, device, [0.4, 0.7])

> **Clinical threshold selection**
>
> Different thresholds optimize for different clinical scenarios. Lower thresholds (0.4) maximize sensitivity for screening, while higher thresholds (0.7) balance precision and recall for diagnostic confirmation.

## Step 5: Profile baseline model for latency, throughput, and memory usage

Comprehensive performance profiling forms the foundation of our optimization strategy. We will measure all key metrics that impact multi-tenant deployment.

In [ ]:
# Initialize performance profiler
profiler = PerformanceProfiler(device=str(device))
print(f"Performance profiler initialized for {device}")

# Get sample batch for profiling
sample_images, sample_labels = get_sample_batch(val_loader)
sample_images = sample_images.to(device)
sample_labels = sample_labels.to(device)

print(f"\nSample batch for profiling:")
print(f"   Batch shape: {sample_images.shape}")
print(f"   Memory usage: {sample_images.numel() * sample_images.element_size() / 1024**2:.1f} MB")

In [ ]:
# Profile inference timing
print("Profiling inference timing...")

timing_results = profiler.profile_inference_time(
    model=baseline_model,
    input_tensor=sample_images,
    num_runs=100,       # Sufficient for statistical significance
    warmup_runs=10      # GPU warmup for consistent measurements
)

print(f"\nTiming Results:")
print(f"   Single Sample Latency: {timing_results['single_sample_ms']:.2f} ms")
print(f"   (Single) Batch Throughput: {timing_results['throughput_samples_per_sec']:.1f} samples/sec")
print(f"   Batch Latency: {timing_results['batch_total_ms']:.2f} ms")
print(f"   Batch Throughput: {timing_results['batch_throughput_samples_per_sec']:.1f} samples/sec")
print(f"   Mean Inference Time: {timing_results['mean_ms']:.2f} ms")
print(f"   95th Percentile: {timing_results['p95_ms']:.2f} ms")
print(f"   Standard Deviation: {timing_results['std_ms']:.2f} ms")

> **Latency vs throughput trade-offs**
>
> Single sample latency measures real-time diagnostic speed, while batch throughput indicates multi-tenant efficiency. Both metrics are crucial for different deployment scenarios.

In [ ]:
# Profile FLOPs for computational efficiency analysis
flops_results = profiler.profile_flops(
    model=baseline_model, 
    input_tensor=sample_images
)

if 'error' in flops_results:
    print(f"FLOPs calculation failed: {flops_results['error']}")
else:
    print(f"\nFLOPs Results:")
    print(f"   Total: {flops_results['total_gflops']:.2f} GFLOPs")
    print(f"   Per Sample: {flops_results['gflops_per_sample']:.2f} GFLOPs")
    if 'module_percentage' in flops_results and flops_results['module_percentage']:
        print(f"\n   Top Operations (by FLOPs):")
        for module, percentage in list(flops_results['module_percentage'].items())[:5]:
            gflops = flops_results['module_breakdown_gflops'][module]
            print(f"     {module}: {percentage:.1f}% ({gflops:.2f} GFLOPs)")

In [ ]:
# Profile GPU memory usage
print("Profiling GPU memory usage...")

memory_results = profiler.profile_memory_usage(
    model=baseline_model,
    input_tensor=sample_images
)

if 'error' not in memory_results:
    print(f"\nMemory Results:")
    print(f"   Peak GPU Memory: {memory_results['peak_memory_mb']:.1f} MB")
    print(f"   Memory Increase: {memory_results['memory_increase_mb']:.1f} MB")
    
    # Component breakdown
    if 'component_breakdown' in memory_results:
        components = memory_results['component_breakdown']
        print(f"\nMemory Component Breakdown:")
        for component, usage in components.items():
            print(f"   {component.replace('_', ' ').title()}: {usage:.1f} MB")
else:
    print(f"WARNING: Memory profiling error: {memory_results['error']}")
    memory_results = {}

> **_Did you notice? A major optimization opportunity hiding in plain sight!_**
> 
> Look carefully at the model summary above. Something doesn't add up with our input/output dimensions...
> 
> Compare the input size we are feeding (64x64) with the first convolution layer's output size. The first Conv2d layer shows output `[-1, 64, 112, 112]` but our input is only 64x64. Where are those extra pixels coming from? Complete the TODO below to find out.
> 
> *Optimization opportunity:* This might be your biggest single optimization win, in both speed and memory usage! Keep this insight in mind as you analyze the profiling results below.

In [ ]:
# Inspect the baseline model's forward method using `inspect.get_source()`
print("Manually inspect the ResNetBaseline forward method:")
baseline_model_forward = inspect.getsource(type(baseline_model).forward)
print(baseline_model_forward)

print("\nDiscussion questions:")
print("1. What happens when height != self.target_size? What is self.target_size set to?") 
print("2. How much computational and memory overhead does F.interpolate add? (Hint: Compare 64x64 vs 224x224 pixel counts)")
print("3. Is this interpolation necessary for pneumonia detection, or just a legacy from ImageNet pretraining?")

In [ ]:
# Detailed PyTorch profiler
print("Running detailed PyTorch profiler...")

detailed_results = profiler.profile_with_pytorch_profiler(
    model=baseline_model,
    input_tensor=sample_images,
    num_steps=10        # Sufficient for operation breakdown analysis
)

if 'error' not in detailed_results:
    print(f"\nOperation Breakdown:")
    op_breakdown = detailed_results['operation_breakdown']
    
    # Show top operations for optimization targeting
    sorted_ops = sorted(op_breakdown.items(), key=lambda x: x[1], reverse=True)
    for op_type, percentage in sorted_ops:
        if percentage > 1:  # Only show operations > 1%
            print(f"   {op_type.replace('_', ' ').title()}: {percentage:.1f}%")
else:
    print(f"WARNING: Detailed profiling error: {detailed_results['error']}")
    detailed_results = {}

## Step 6: Visualize and save baseline model's performance

Comprehensive visualization helps understand optimization opportunities and track progress across optimization experiments.

In [ ]:
# Visualize performance profile
plot_performance_profile(timing_results)

# Visualize operation breakdown
if detailed_results and 'operation_breakdown' in detailed_results:
    plot_operation_breakdown(detailed_results['operation_breakdown'])
else:
    print("WARNING: Operation breakdown visualization not available")

In [ ]:
# Compile baseline results for optimization notebooks
baseline_results = {
    'model_name': 'ResNet-18 Baseline',
    'architecture': model_info['architecture'],
    'total_parameters': model_info['total_parameters'],
    'model_size_mb': model_info['model_size_mb'],
    'config': CONFIG,
    'eval_results': eval_results,
    'timing': timing_results,
    'flops': flops_results,
    'memory': memory_results,
    'operation_breakdown': detailed_results['operation_breakdown'],
    'model_info': model_info,
    'dataset_info': dataset_info,
    'parameter_breakdown': count_parameters_by_type(baseline_model)
}

# Save baseline results
with open('../results/baseline_results.pkl', 'wb') as f:
    pickle.dump(baseline_results, f)

print("Baseline results saved to 'baseline_results.pkl' in the `results/` folder")
print("   This will be used for comparison in optimization notebooks.")

> **Brainstorming time!**
> 
> Based on your profiling results above, analyze the following:
> 
> 1. **Primary bottleneck**: What is the main performance bottleneck - compute time, memory usage, or data transfer?
> 
> 2. **Operation analysis**: Which types of operations (convolution, linear, activation) consume the most time? What percentage?
> 
> 3. **Memory patterns**: How much memory does the model use during inference? What contributes most to memory usage?
> 
> 4. **Optimization priority**: Based on the profiling data, which optimization techniques would you prioritize:
>    - Architecture modifications (channel reduction, efficient blocks)
>    - Precision optimization (mixed precision, quantization)
>    - Hardware acceleration (TensorRT for GPU, ...)
>   
>    _IMPORTANT:_ Did you discover the major inefficiency we hinted at earlier? How much improvement could removing the 64→224 interpolation provide?

---

## Step 7: Analyze optimization opportunities

Now that we have established the baseline performance, it's time for you to conduct a deeper analysis that will guide the optimization strategy. This section contains **two focused analysis checkpoints**:

1. **Architecture optimization analysis** - Identify specific opportunities in the ResNet-18 architecture
2. **Deployment optimization analysis** - Understand hardware acceleration and deployment strategies

Complete these analysis checkpoints to develop your optimization roadmap!

### Analysis checkpoint 1: Architecture optimization opportunities

**Task:** Analyze the ResNet-18 architecture to identify the **top 2 optimization opportunities** from the techniques covered in the course. Available techniques to consider include:
- Grouped convolutions
- Depth-wise separable convolutions
- Inverted residuals with linear bottlenecks
- Low-rank factorization
- Channel organization strategies _(NOTE: this is a hybrid optimization between architecture and hardware)_
- Parameter sharing / weight tying

Feel free to skip programmatic analysis of techniques which you deem to be non-applicable or less performant, but provide an explanation here or in the notebook's final markdown cell.

**IMPORTANT:** Don't forget to also analyze the potential of interpolation removal from the model's forward method!

#### Recommended strategic analysis approach

Calculate the expected impact of applying each technique on parameter reduction programmatically to simplify follow-up analysis. Consider how parameter reduction / architectural improvements for each technique correlate with memory size (activation vs parameters), FLOPs, latency, throughput, and sensitivity to estimate optimization opportunity. HINTS are in each function's signature.

To populate the analysis dictionary with estimated optimiation opportunity, you can either:
1. Programmatically calculate optimization opportunity from parameter reduction and rule-of-thumb
2. Directly add the expected value in the analysis entries, and add a one-line explanation of the value selected as an in-line comment.

In [ ]:
## Implement logic for each single technique analysis

def _get_conv_coverage_from_profiler():
    """Return convolution time coverage (0-1) from detailed_results, robust to key naming."""
    try:
        ops = detailed_results.get('operation_breakdown', {})
        conv_pct = 0.0
        for k, v in ops.items():
            kl = k.replace('_', ' ').lower()
            if 'convolution' in kl or 'conv' in kl:
                conv_pct += float(v)
        return conv_pct / 100.0 if conv_pct > 0 else 0.0
    except Exception:
        return 0.0

def analyze_grouped_conv_potential(model, sample_input_shape=(3, CONFIG["image_size"], CONFIG["image_size"]), groups=2):
    """
    Identify convolution layers suitable for grouped convolutions and estimate impact.
    Assumptions: FLOPs scale ~ linearly with parameter count per conv; overall
    speedup estimated via Amdahl's law using conv coverage from the profiler.
    """
    import torch.nn as nn

    conv_total_params = 0
    candidate_params = 0
    candidates = []

    for name, m in model.named_modules():
        if isinstance(m, nn.Conv2d) and m.kernel_size[0] > 1 and m.groups == 1:
            params = m.in_channels * m.out_channels * (m.kernel_size[0] ** 2)
            conv_total_params += params
            if m.in_channels % groups == 0 and m.out_channels % groups == 0:
                candidate_params += params
                candidates.append({
                    'name': name,
                    'in': m.in_channels,
                    'out': m.out_channels,
                    'k': m.kernel_size[0],
                    'stride': m.stride[0]
                })

    if conv_total_params == 0:
        conv_total_params = 1

    # Param/FLOP reduction on candidates
    reduction_ratio_layer = 1.0 - (1.0 / groups)
    param_reduction_ratio_overall = (candidate_params / conv_total_params) * reduction_ratio_layer

    conv_cov = _get_conv_coverage_from_profiler() or 0.88
    candidate_cov = candidate_params / conv_total_params

    # Amdahl’s law across conv + non-conv fractions
    new_time = (1 - conv_cov) + conv_cov * (candidate_cov * (1 / groups) + (1 - candidate_cov) * 1.0)
    speedup = 1.0 / new_time if new_time > 0 else 1.0

    # Parameter memory reduction (MB)
    saved_params = candidate_params * reduction_ratio_layer
    saved_mb = (saved_params * 4) / (1024 ** 2)

    base_throughput = timing_results.get('batch_throughput_samples_per_sec', timing_results.get('throughput_samples_per_sec', 0))
    est_throughput = base_throughput * speedup

    analysis = {
        'technique': 'Grouped Convolutions',
        'groups': groups,
        'candidate_layers': candidates,
        'total_candidates': len(candidates),
        'avg_flop_reduction_percent': round(param_reduction_ratio_overall * 100, 2),
        'avg_param_reduction_percent': round(param_reduction_ratio_overall * 100, 2),
        'estimated_speedup': round(speedup, 3),
        'estimated_memory_reduction_mb': round(saved_mb, 2),
        'estimated_throughput_samples_sec': round(est_throughput, 1),
        'throughput_improvement_percent': round((speedup - 1) * 100, 1),
        'sensitivity_risk': 'Moderate (may reduce cross-channel mixing; validate sensitivity)'
    }

    return analysis

def analyze_depthwise_separable_potential(model, sample_input_shape=(3, CONFIG["image_size"], CONFIG["image_size"])):
    """
    Estimate impact of replacing eligible 3×3 convs with depthwise-separable convs.
    Separable params = k^2*in + in*out. Standard = k^2*in*out.
    """
    import torch.nn as nn

    conv_total_params = 0
    separable_params_total = 0
    standard_params_total = 0
    candidates = []

    for name, m in model.named_modules():
        if isinstance(m, nn.Conv2d) and m.kernel_size[0] > 1 and m.groups == 1 and m.in_channels >= 16:
            k = m.kernel_size[0]
            std = m.in_channels * m.out_channels * (k ** 2)
            sep = (m.in_channels * (k ** 2)) + (m.in_channels * m.out_channels)
            standard_params_total += std
            separable_params_total += sep
            candidates.append({'name': name, 'in': m.in_channels, 'out': m.out_channels, 'k': k, 'stride': m.stride[0]})
        if isinstance(m, nn.Conv2d) and m.kernel_size[0] > 1:
            conv_total_params += m.in_channels * m.out_channels * (m.kernel_size[0] ** 2)

    if conv_total_params == 0 or standard_params_total == 0:
        speedup = 1.0
        avg_reduction = 0.0
        saved_mb = 0.0
    else:
        # Overall conv time fraction and candidate coverage
        conv_cov = _get_conv_coverage_from_profiler() or 0.88
        candidate_cov = standard_params_total / conv_total_params

        # Conv time factor after separable substitution on candidates
        conv_factor = candidate_cov * (separable_params_total / standard_params_total) + (1 - candidate_cov) * 1.0
        theoretical_speedup = 1.0 / ((1 - conv_cov) + conv_cov * conv_factor)
        # Penalize for memory-access overhead (empirical)
        speedup = 1.0 + (theoretical_speedup - 1.0) * 0.6

        avg_reduction = 1.0 - (separable_params_total / standard_params_total)
        saved_params = (standard_params_total - separable_params_total)
        saved_mb = (saved_params * 4) / (1024 ** 2)

    base_throughput = timing_results.get('batch_throughput_samples_per_sec', timing_results.get('throughput_samples_per_sec', 0))
    est_throughput = base_throughput * speedup

    analysis = {
        'technique': 'Depthwise Separable Convolutions',
        'candidate_layers': candidates,
        'total_candidates': len(candidates),
        'avg_flop_reduction_percent': round(avg_reduction * 100, 2),
        'avg_param_reduction_percent': round(avg_reduction * 100, 2),
        'estimated_speedup': round(speedup, 3),
        'estimated_memory_reduction_mb': round(saved_mb, 2),
        'estimated_throughput_samples_sec': round(est_throughput, 1),
        'throughput_improvement_percent': round((speedup - 1) * 100, 1),
        'sensitivity_risk': 'Moderate (structure change; retrain/validate)'
    }

    return analysis

def analyze_inverted_residuals_potential(model):
    """
    Heuristic: consider stride-2 transition blocks (where channels double)
    as candidates for MobileNetV2-style inverted residuals.
    """
    import torch.nn as nn
    conv_total_params = 0
    trans_params = 0
    trans_layers = []
    for name, m in model.named_modules():
        if isinstance(m, nn.Conv2d) and m.kernel_size[0] > 1:
            conv_total_params += m.in_channels * m.out_channels * (m.kernel_size[0] ** 2)
            if m.stride[0] == 2 and m.groups == 1:
                trans_params += m.in_channels * m.out_channels * (m.kernel_size[0] ** 2)
                trans_layers.append({'name': name, 'in': m.in_channels, 'out': m.out_channels, 'k': m.kernel_size[0]})

    conv_cov = _get_conv_coverage_from_profiler() or 0.88
    candidate_cov = (trans_params / conv_total_params) if conv_total_params else 0.0
    # Assume ~1.2x faster on those transitions after IR blocks
    conv_factor = candidate_cov * (1 / 1.2) + (1 - candidate_cov) * 1.0
    new_time = (1 - conv_cov) + conv_cov * conv_factor
    speedup = 1.0 / new_time if new_time > 0 else 1.0

    analysis = {
        'technique': 'Inverted Residuals',
        'transition_layers': trans_layers,
        'total_candidates': len(trans_layers),
        'estimated_speedup': round(speedup, 3),
        'avg_flop_reduction_percent': round((1 - conv_factor) * conv_cov * 100, 2),
        'estimated_memory_reduction_mb': 0.0,
        'estimated_throughput_samples_sec': round(timing_results.get('batch_throughput_samples_per_sec', 0) * speedup, 1),
        'throughput_improvement_percent': round((speedup - 1) * 100, 1),
        'sensitivity_risk': 'Low‑Moderate (structure change in downsampling blocks)'
    }
    return analysis

def analyze_lowrank_factorization_potential(model, batch_size=32):
    """
    Lightweight estimate for low‑rank factorization potential (linear layers).
    ResNet‑18 has a tiny classifier (512→2), so impact is negligible.
    """
    import torch.nn as nn
    total_linear = 0
    linear_layers = 0
    saved = 0
    for m in model.modules():
        if isinstance(m, nn.Linear):
            linear_layers += 1
            params = m.in_features * m.out_features
            total_linear += params
            # hypothetical rank‑reduction to 1/4
            r = max(1, min(m.in_features, m.out_features) // 4)
            sep = m.in_features * r + r * m.out_features
            saved += max(0, params - sep)
    total_params = sum(p.numel() for p in model.parameters())
    saved_mb = (saved * 4) / (1024 ** 2)
    speedup = 1.0 + min(0.02, saved / max(1, total_params))  # tiny effect
    base_tput = timing_results.get('batch_throughput_samples_per_sec', 0)
    return {
        'technique': 'Low‑Rank Factorization (FC)',
        'total_candidates': linear_layers,
        'avg_param_reduction_percent': round((saved / max(1, total_linear)) * 100, 3) if total_linear else 0.0,
        'estimated_speedup': round(speedup, 3),
        'avg_flop_reduction_percent': round((saved / max(1, total_params)) * 100, 3),
        'estimated_memory_reduction_mb': round(saved_mb, 3),
        'estimated_throughput_samples_sec': round(base_tput * speedup, 1),
        'throughput_improvement_percent': round((speedup - 1) * 100, 2),
        'sensitivity_risk': 'Low (classifier only)'
    }

def analyze_channel_organization_potential(model):
    """Estimate win from channels_last + in‑place activations (no FLOP change)."""
    import torch.nn as nn
    relu_not_inplace = sum(1 for m in model.modules() if isinstance(m, nn.ReLU) and not m.inplace)
    base_speedup = 1.08  # conservative channels_last benefit
    speedup = base_speedup * (1.0 + min(relu_not_inplace, 20) * 0.008)
    base_tput = timing_results.get('batch_throughput_samples_per_sec', timing_results.get('throughput_samples_per_sec', 0))
    analysis = {
        'technique': 'Channel Organization (channels_last + in-place ReLU)',
        'inplace_relu_candidates': relu_not_inplace,
        'inplace_opportunities': relu_not_inplace,
        'estimated_speedup': round(speedup, 3),
        'estimated_memory_reduction_mb': 0,
        'avg_flop_reduction_percent': 0,
        'estimated_throughput_samples_sec': round(base_tput * speedup, 1),
        'throughput_improvement_percent': round((speedup - 1) * 100, 1),
        'sensitivity_risk': 'Low (no numerical change expected)'
    }
    return analysis

def analyze_parameter_sharing_potential(model):
    """Rough estimate of weight sharing potential across identical conv shapes."""
    import torch.nn as nn
    from collections import defaultdict
    shape_groups = defaultdict(list)
    total_params = 0
    for name, m in model.named_modules():
        if isinstance(m, nn.Conv2d) and m.groups == 1:
            k = m.kernel_size[0]
            params = m.in_channels * m.out_channels * (k ** 2)
            total_params += params
            key = (m.in_channels, m.out_channels, k)
            shape_groups[key].append((name, params))
    shareable = 0
    groups_summary = {}
    for key, items in shape_groups.items():
        if len(items) > 1:
            saved = sum(p for _, p in items[1:])
            shareable += saved
            groups_summary[str(key)] = len(items)
    sharing_potential = min(0.25, (shareable / total_params) if total_params else 0.0)
    speedup = 1.0 + sharing_potential * 0.4
    saved_mb = (shareable * 4) / (1024 ** 2)
    base_tput = timing_results.get('batch_throughput_samples_per_sec', 0)
    analysis = {
        'technique': 'Parameter Sharing',
        'similar_layer_groups': groups_summary,
        'sharing_potential_percent': round(sharing_potential * 100, 2),
        'avg_flop_reduction_percent': round(sharing_potential * 100, 2),
        'estimated_speedup': round(speedup, 3),
        'estimated_memory_reduction_mb': round(saved_mb, 2),
        'estimated_throughput_samples_sec': round(base_tput * speedup, 1),
        'throughput_improvement_percent': round((speedup - 1) * 100, 1),
        'sensitivity_risk': 'Medium (may constrain representation; requires retraining)'
    }
    return analysis

def analyze_interpolation_removal_potential(model, sample_input_shape=(3, CONFIG["image_size"], CONFIG["image_size"])):
    """Quantify benefit of removing 64→224 interpolation inside forward()."""
    target = getattr(model, 'target_size', 224)
    native = CONFIG["image_size"]
    factor = (target / native) ** 2 if native else 1.0
    conv_cov = _get_conv_coverage_from_profiler() or 0.88
    scalable_portion = conv_cov * 0.6  # heuristic from instructions
    fixed_portion = 1.0 - scalable_portion
    new_time = fixed_portion + scalable_portion / factor
    speedup = 1.0 / new_time if new_time > 0 else 1.0
    flop_reduction_pct = (1.0 - 1.0 / factor) * conv_cov * 100

    # Estimate activation memory savings
    comp = (memory_results or {}).get('component_breakdown', {}) if isinstance(memory_results, dict) else {}
    act_key = 'estimated_activations_mb' if 'estimated_activations_mb' in comp else ('activations_mb' if 'activations_mb' in comp else None)
    saved_mb = 0.0
    if act_key:
        saved_mb = comp[act_key] * (1.0 - 1.0 / factor)

    base_tput = timing_results.get('batch_throughput_samples_per_sec', timing_results.get('throughput_samples_per_sec', 0))
    analysis = {
        'technique': 'Interpolation Removal (Native Resolution)',
        'interpolation_size': int(target),
        'original_image_size': native,
        'avg_flop_reduction_percent': round(flop_reduction_pct, 2),
        'estimated_speedup': round(speedup, 3),
        'estimated_memory_reduction_mb': round(saved_mb, 2),
        'estimated_throughput_samples_sec': round(base_tput * speedup, 1),
        'throughput_improvement_percent': round((speedup - 1) * 100, 1),
        'sensitivity_risk': 'Low‑Moderate (must validate at native resolution)'
    }

    return analysis





In [ ]:
# Run the complete architecture optimization analysis with all 5 implemented analysis functions above
def run_architecture_optimization_analysis(model):
    """
    Main function to run all architecture optimization analyses.
    """
    print("Running architecture optimization analysis...")
    
    # Call each analysis function
    depthwise_analysis = analyze_depthwise_separable_potential(model)
    grouped_analysis = analyze_grouped_conv_potential(model)
    inverted_analysis = analyze_inverted_residuals_potential(model)
    lowrank_analysis = analyze_lowrank_factorization_potential(model)
    channel_analysis = analyze_channel_organization_potential(model)
    sharing_analysis = analyze_parameter_sharing_potential(model)
    interpolation_analysis = analyze_interpolation_removal_potential(model)
    
    # Combine all analyses into comprehensive result
    # TODO: Remove any techniques which haven't been analyzed
    optimization_techniques = {
        'grouped_convolutions': {
            'candidate_layers': grouped_analysis['total_candidates'],
            'groups': grouped_analysis['groups'],
            'avg_param_reduction_percent': grouped_analysis['avg_param_reduction_percent'],
            'estimated_speedup': grouped_analysis['estimated_speedup'],
            'avg_flop_reduction_percent': grouped_analysis['avg_flop_reduction_percent'],
            'estimated_memory_reduction_mb': grouped_analysis['estimated_memory_reduction_mb'],
            'throughput_improvement_percent': grouped_analysis['throughput_improvement_percent'],
            'sensitivity_risk': grouped_analysis['sensitivity_risk'],
            'details': grouped_analysis['candidate_layers'][:3]  # Show top 3
        },
        'depthwise_separable': {
            'candidate_layers': depthwise_analysis['total_candidates'],
            'avg_param_reduction_percent': depthwise_analysis['avg_param_reduction_percent'],
            'estimated_speedup': depthwise_analysis['estimated_speedup'],
            'avg_flop_reduction_percent': depthwise_analysis['avg_flop_reduction_percent'],
            'estimated_memory_reduction_mb': depthwise_analysis['estimated_memory_reduction_mb'],
            'throughput_improvement_percent': depthwise_analysis['throughput_improvement_percent'],
            'sensitivity_risk': depthwise_analysis['sensitivity_risk'],
            'details': depthwise_analysis['candidate_layers'][:3]  # Show top 3
        },
        'inverted_residuals': {
            'expansion_candidates': inverted_analysis['total_candidates'],
            'estimated_speedup': inverted_analysis['estimated_speedup'],
            'avg_flop_reduction_percent': inverted_analysis['avg_flop_reduction_percent'],
            'estimated_memory_reduction_mb': inverted_analysis['estimated_memory_reduction_mb'],
            'throughput_improvement_percent': inverted_analysis['throughput_improvement_percent'],
            'sensitivity_risk': inverted_analysis['sensitivity_risk']
        },
        'low_rank_factorization': {
            'candidate_layers': lowrank_analysis['total_candidates'],
            'avg_param_reduction_percent': lowrank_analysis['avg_param_reduction_percent'],
            'estimated_speedup': lowrank_analysis['estimated_speedup'],
            'avg_flop_reduction_percent': lowrank_analysis['avg_flop_reduction_percent'],
            'estimated_memory_reduction_mb': lowrank_analysis['estimated_memory_reduction_mb'],
            'throughput_improvement_percent': lowrank_analysis['throughput_improvement_percent'],
            'sensitivity_risk': lowrank_analysis['sensitivity_risk']
        },
        'channel_organization': {
            'inplace_opportunities': channel_analysis['inplace_opportunities'],
            'estimated_speedup': channel_analysis['estimated_speedup'],
            'avg_flop_reduction_percent': channel_analysis['avg_flop_reduction_percent'],
            'estimated_memory_reduction_mb': channel_analysis['estimated_memory_reduction_mb'],
            'throughput_improvement_percent': channel_analysis['throughput_improvement_percent'],
            'sensitivity_risk': channel_analysis['sensitivity_risk']
        },
        'parameter_sharing': {
            'sharing_potential_percent': sharing_analysis['sharing_potential_percent'],
            'estimated_speedup': sharing_analysis['estimated_speedup'],
            'avg_flop_reduction_percent': sharing_analysis['avg_flop_reduction_percent'],
            'estimated_memory_reduction_mb': sharing_analysis['estimated_memory_reduction_mb'],
            'throughput_improvement_percent': sharing_analysis['throughput_improvement_percent'],
            'sensitivity_risk': sharing_analysis['sensitivity_risk']
        },
        'interpolation_removal': {
            'current_input_size': interpolation_analysis["interpolation_size"],
            'native_input_size': interpolation_analysis["original_image_size"],
            'estimated_speedup': interpolation_analysis['estimated_speedup'],
            'avg_flop_reduction_percent': interpolation_analysis['avg_flop_reduction_percent'],
            'estimated_memory_reduction_mb': interpolation_analysis['estimated_memory_reduction_mb'],
            'throughput_improvement_percent': interpolation_analysis['throughput_improvement_percent'],
            'sensitivity_risk': interpolation_analysis['sensitivity_risk']
        }
    }
    
    return optimization_techniques

# Execute architecture analysis
arch_analysis = run_architecture_optimization_analysis(baseline_model)

# Display results
print(f"\nARCHITECTURE OPTIMIZATION RESULTS:")
for technique, details in arch_analysis.items():
    print(f"\n   {technique.replace('_', ' ').title()}:")
    for key, value in details.items():
        if key != 'details':
            print(f"     {key.replace('_', ' ').title()}: {value}")

In [ ]:
# Summarize architecture techniques quantitatively
import pandas as pd
rows=[]
for name, d in arch_analysis.items():
    # robust candidate counting across techniques
    cand = 0
    if isinstance(d.get('candidate_layers'), list):
        cand = len(d.get('candidate_layers'))
    elif isinstance(d.get('transition_layers'), list):
        cand = len(d.get('transition_layers'))
    elif isinstance(d.get('expansion_candidates'), (int, float)):
        cand = int(d.get('expansion_candidates'))
    elif isinstance(d.get('inplace_opportunities'), (int, float)):
        cand = int(d.get('inplace_opportunities'))

    rows.append({
        'technique': name,
        'candidates': cand,
        'avg_param_reduction_%': d.get('avg_param_reduction_percent', 0),
        'avg_FLOP_reduction_%': d.get('avg_flop_reduction_percent', 0),
        'estimated_speedup_x': d.get('estimated_speedup', 1.0),
        'throughput_improvement_%': d.get('throughput_improvement_percent', 0),
        'mem_savings_MB': d.get('estimated_memory_reduction_mb', 0),
        'risk': d.get('sensitivity_risk','')
    })
arch_df=pd.DataFrame(rows)
arch_df.sort_values(by='estimated_speedup_x', ascending=False, inplace=True)
arch_df.reset_index(drop=True, inplace=True)
arch_df



> **_Before you move on...brainstorming time!_**
> 
> Based on your architecture analysis results, collect yours insights on architectural opportunities
> 
> 1. **Primary bottleneck**: What operation type consumes the most compute time in your model? Why does this pattern make sense for ResNet-18 architecture? How does this inform your optimization strategy?
> 
> 2. **Resource trade-offs:**: Which optimizations reduce parameters vs which improve compute efficiency? How might accuracy be affected by each technique?
> 
> 3. **Optimization priority**: In which order would you recommend implementing the 5 architectural techniques? Consider both impact and implementation difficulty.
> 
> 4. **Feasibility assessment:**: Will the combined optimizations achieve the 3ms target? If not, what additional techniques might be needed?

### Implement logic for each single technique analysis 

def analyze_mixed_precision_potential(detailed_results):
    """
    Analyze mixed-precision (FP16) potential using profiler op breakdown + simple heuristics.
    Returns a dict with coverage, speedup estimate, memory reduction and throughput impact.
    """
    ops = (detailed_results or {}).get('operation_breakdown', {})
    # Sum percentages of conv/matmul-like ops as tensor-core eligible
    eligible = 0.0
    total = 0.0
    for k, v in ops.items():
        total += float(v)
        kl = k.replace('_', ' ').lower()
        if any(t in kl for t in ['convolution', 'conv', 'matmul', 'gemm', 'linear']):
            eligible += float(v)
    coverage = (eligible / total * 100.0) if total > 0 else 70.0  # default if missing

    # Speedup model: baseline 1.5x; if coverage >50%, add small slope toward 2.5x
    if coverage >= 50:
        estimated_speedup = 1.8 + (coverage - 50) * 0.014  # 50%→1.8x, 100%→2.5x
    else:
        estimated_speedup = 1.5
    estimated_speedup = float(max(1.0, min(2.5, estimated_speedup)))

    # Memory savings: FP32→FP16 ~50% for activations + params
    peak_mb = 0.0
    if isinstance(memory_results, dict):
        peak_mb = float(memory_results.get('peak_memory_mb', 0.0))
    mem_savings = 0.5 * peak_mb if peak_mb else 0.0

    base_tput = timing_results.get('batch_throughput_samples_per_sec', timing_results.get('throughput_samples_per_sec', 0))
    throughput = base_tput * estimated_speedup if base_tput else 0

    return {
        'technique': 'Mixed Precision (FP16)',
        'mixed_precision_coverage_percent': float(coverage),
        'avg_flop_reduction_percent': float(min(50.0, coverage)),
        'estimated_speedup': float(estimated_speedup),
        'estimated_memory_reduction_mb': float(mem_savings),
        'estimated_throughput_samples_sec': float(throughput),
        'throughput_improvement_percent': float((estimated_speedup - 1) * 100),
        'sensitivity_risk': 'Low–Moderate (validate numerics and calibration)'
    }

def analyze_batch_processing_scenarios(model, mixed_precision_speedup, sample_input_shape=(3, CONFIG["image_size"], CONFIG["image_size"])):
    """
    Profile multiple batch sizes and propose optimal configs for real-time and throughput.
    Uses the global `profiler` created earlier and the observed `sample_images` shape.
    """
    # Choose a reasonable range up to the observed sample batch size
    max_bs = int(sample_images.shape[0]) if 'sample_images' in globals() else 32
    batch_sizes = [b for b in [1, 2, 4, 8, 16, 32] if b <= max_bs]

    print("   Profiling multiple batch sizes...")
    batch_results = profiler.profile_multiple_batch_sizes(
        model, sample_images.shape, batch_sizes
    )

    # Parse results and pick optimal batch sizes
    valid = {}
    for k, v in batch_results.items():
        if isinstance(v, dict) and 'timing' in v:
            t = v['timing']
            per_sample = t.get('batch_per_sample_ms', t.get('single_sample_ms', None))
            throughput = t.get('batch_throughput_samples_per_sec', t.get('throughput_samples_per_sec', None))
            if per_sample is not None and throughput is not None:
                valid[int(k.split('_')[1])] = {
                    'latency_ms': float(per_sample),
                    'throughput': float(throughput)
                }

    scenarios = {
        'real_time_diagnosis': {
            'optimal_batch_size': None,
            'current_latency_ms': None,
            'mixed_precision_latency_ms': None,
            'use_case': 'Emergency diagnosis, single patient processing'
        },
        'batch_processing': {
            'optimal_batch_size': None,
            'current_throughput_samples_sec': None,
            'mixed_precision_throughput_samples_sec': None,
            'use_case': 'Screening workflows, research processing'
        }
    }

    if valid:
        # minimize latency per sample for real-time
        rt_bs = min(valid.keys(), key=lambda b: valid[b]['latency_ms'])
        rt_lat = valid[rt_bs]['latency_ms']
        scenarios['real_time_diagnosis'].update({
            'optimal_batch_size': rt_bs,
            'current_latency_ms': round(rt_lat, 2),
            'mixed_precision_latency_ms': round(rt_lat / mixed_precision_speedup, 2)
        })
        # maximize throughput for batch processing
        th_bs = max(valid.keys(), key=lambda b: valid[b]['throughput'])
        th_tp = valid[th_bs]['throughput']
        scenarios['batch_processing'].update({
            'optimal_batch_size': th_bs,
            'current_throughput_samples_sec': round(th_tp, 1),
            'mixed_precision_throughput_samples_sec': round(th_tp * mixed_precision_speedup, 1)
        })

    if not valid:
        # Fallback using existing timing_results
        rt_lat = float(timing_results.get('single_sample_ms', timing_results.get('mean_ms', 0)))
        base_tp = float(timing_results.get('batch_throughput_samples_per_sec', timing_results.get('throughput_samples_per_sec', 0)))
        scenarios['real_time_diagnosis'].update({
            'optimal_batch_size': 1,
            'current_latency_ms': round(rt_lat, 2) if rt_lat else None,
            'mixed_precision_latency_ms': round(rt_lat / mixed_precision_speedup, 2) if rt_lat else None
        })
        scenarios['batch_processing'].update({
            'optimal_batch_size': timing_results.get('batch_size', 1),
            'current_throughput_samples_sec': round(base_tp, 1) if base_tp else None,
            'mixed_precision_throughput_samples_sec': round(base_tp * mixed_precision_speedup, 1) if base_tp else None
        })
    return batch_results, scenarios




In [ ]:
### Implement logic for each single technique analysis 

def analyze_mixed_precision_potential(detailed_results):
    """
    Analyze mixed-precision (FP16) potential using profiler op breakdown + simple heuristics.
    Returns a dict with coverage, speedup estimate, memory reduction and throughput impact.
    """
    ops = (detailed_results or {}).get('operation_breakdown', {})
    eligible = 0.0
    total = 0.0
    for k, v in ops.items():
        total += float(v)
        kl = k.replace('_', ' ').lower()
        if any(t in kl for t in ['convolution', 'conv', 'matmul', 'gemm', 'linear']):
            eligible += float(v)
    coverage = (eligible / total * 100.0) if total > 0 else ( (_get_conv_coverage_from_profiler()*100.0) if 'detailed_results' in globals() else 70.0 )

    if coverage >= 50:
        estimated_speedup = 1.8 + (coverage - 50.0) * 0.014
    else:
        estimated_speedup = 1.5
    estimated_speedup = float(max(1.0, min(2.5, estimated_speedup)))

    peak_mb = 0.0
    if isinstance(memory_results, dict):
        peak_mb = float(memory_results.get('peak_memory_mb', 0.0))
    mem_savings = 0.5 * peak_mb if peak_mb else 0.0

    base_tput = timing_results.get('batch_throughput_samples_per_sec', timing_results.get('throughput_samples_per_sec', 0))
    throughput = base_tput * estimated_speedup if base_tput else 0

    return {
        'technique': 'Mixed Precision (FP16)',
        'mixed_precision_coverage_percent': float(coverage),
        'avg_flop_reduction_percent': float(min(50.0, coverage)),
        'estimated_speedup': float(estimated_speedup),
        'estimated_memory_reduction_mb': float(mem_savings),
        'estimated_throughput_samples_sec': float(throughput),
        'throughput_improvement_percent': float((estimated_speedup - 1) * 100),
        'sensitivity_risk': 'Low–Moderate (validate numerics and calibration)'
    }

def analyze_batch_processing_scenarios(model, mixed_precision_speedup, sample_input_shape=(3, CONFIG["image_size"], CONFIG["image_size"])):
    """
    Profile multiple batch sizes and propose optimal configs for real-time and throughput.
    Uses the global `profiler` created earlier and the observed `sample_images` shape.
    """
    max_bs = int(sample_images.shape[0]) if 'sample_images' in globals() else 32
    batch_sizes = [b for b in [1, 2, 4, 8, 16, 32] if b <= max_bs]

    print("   Profiling multiple batch sizes...")
    batch_results = profiler.profile_multiple_batch_sizes(
        model, sample_images.shape, batch_sizes
    )

    valid = {}
    for k, v in batch_results.items():
        if isinstance(v, dict) and 'timing' in v:
            t = v['timing']
            per_sample = t.get('batch_per_sample_ms', t.get('single_sample_ms', None))
            throughput = t.get('batch_throughput_samples_per_sec', t.get('throughput_samples_per_sec', None))
            if per_sample is not None and throughput is not None:
                valid[int(k.split('_')[1])] = {
                    'latency_ms': float(per_sample),
                    'throughput': float(throughput)
                }

    scenarios = {
        'real_time_diagnosis': {
            'optimal_batch_size': None,
            'current_latency_ms': None,
            'mixed_precision_latency_ms': None,
            'use_case': 'Emergency diagnosis, single patient processing'
        },
        'batch_processing': {
            'optimal_batch_size': None,
            'current_throughput_samples_sec': None,
            'mixed_precision_throughput_samples_sec': None,
            'use_case': 'Screening workflows, research processing'
        }
    }

    if valid:
        rt_bs = min(valid.keys(), key=lambda b: valid[b]['latency_ms'])
        rt_lat = valid[rt_bs]['latency_ms']
        scenarios['real_time_diagnosis'].update({
            'optimal_batch_size': rt_bs,
            'current_latency_ms': round(rt_lat, 2),
            'mixed_precision_latency_ms': round(rt_lat / mixed_precision_speedup, 2)
        })
        th_bs = max(valid.keys(), key=lambda b: valid[b]['throughput'])
        th_tp = valid[th_bs]['throughput']
        scenarios['batch_processing'].update({
            'optimal_batch_size': th_bs,
            'current_throughput_samples_sec': round(th_tp, 1),
            'mixed_precision_throughput_samples_sec': round(th_tp * mixed_precision_speedup, 1)
        })

    if not valid:
        rt_lat = float(timing_results.get('single_sample_ms', timing_results.get('mean_ms', 0)))
        base_tp = float(timing_results.get('batch_throughput_samples_per_sec', timing_results.get('throughput_samples_per_sec', 0)))
        scenarios['real_time_diagnosis'].update({
            'optimal_batch_size': 1,
            'current_latency_ms': round(rt_lat, 2) if rt_lat else None,
            'mixed_precision_latency_ms': round(rt_lat / mixed_precision_speedup, 2) if rt_lat else None
        })
        scenarios['batch_processing'].update({
            'optimal_batch_size': timing_results.get('batch_size', 1),
            'current_throughput_samples_sec': round(base_tp, 1) if base_tp else None,
            'mixed_precision_throughput_samples_sec': round(base_tp * mixed_precision_speedup, 1) if base_tp else None
        })

    return batch_results, scenarios



In [ ]:
# Run the complete deployment optimization analysis with all 2 implemented analysis functions above
def run_deployment_optimization_analysis(baseline_model, timing_results, memory_results, detailed_results):
    """
    Main function to run all deployment optimization analyses.
    """
    print("Running Deployment Optimization Analysis...")

    # Analyze mixed precision potential
    mixed_precision = analyze_mixed_precision_potential(detailed_results)
    mixed_precision_speedup = mixed_precision['estimated_speedup']
    
    # Analyze batch processing scenarios
    batch_results, batch_scenarios = analyze_batch_processing_scenarios(baseline_model, mixed_precision_speedup)

    # Visualize batch size analysis for deployment understanding
    plot_batch_size_comparison(batch_results)

    # Calculate deployment readiness
    current_latency = timing_results['single_sample_ms']
    current_throughput = timing_results['throughput_samples_per_sec']
    
    performance_metrics = {
        'latency_ms': current_latency,
        'throughput_samples_sec': current_throughput
    }
    
    return {
        'mixed_precision': mixed_precision,
        'batch_scenarios': batch_scenarios
    }

# Execute deployment analysis
deployment_analysis = run_deployment_optimization_analysis(baseline_model, timing_results, memory_results, detailed_results)

print(f"\nDEPLOYMENT OPTIMIZATION RESULTS:")
mp_details = deployment_analysis['mixed_precision']
print(f"\n   Mixed Precision (FP16):")
print(f"     Tensor Core Eligible: {mp_details['mixed_precision_coverage_percent']:.1f}%")
print(f"     Estimated FLOP improvements: {mp_details['avg_flop_reduction_percent']:.1f}MB")
print(f"     Estimated Speedup: {mp_details['estimated_speedup']:.1f}x")
print(f"     Estimated Throughput improvements %: {mp_details['throughput_improvement_percent']:.1f}%")
print(f"     Estimated Memory Savings: {mp_details['estimated_memory_reduction_mb']:.1f}MB")
print(f"     Estimated Sensitivity Risk: {mp_details['sensitivity_risk']}")

if 'error' not in deployment_analysis['batch_scenarios']:
    scenarios = deployment_analysis['batch_scenarios']
    print(f"\n   Deployment Scenarios:")
    for scenario, details in scenarios.items():
        print(f"     {scenario.replace('_', ' ').title()}:")
        print(f"       Optimal Batch Size: {details['optimal_batch_size']}")
        print(f"       Use Case: {details['use_case']}")

> **_Before you move on...brainstorming time!_**
> 
> Based on your deployment analysis results above, collect your thoughts about hardware deployment opportunities - this will help you in completing your optimization plan at the end of the notebook:
>  
> 1. **Shared-Resource Constraints**: What is the main limiting factor when deploying the model alongside other applications—memory or compute?
> 
> 2. **Batch processing trade-offs**: How does performance change with batch size? What's the optimal configuration for different deployment scenarios?
> 
> 3. **Mixed precision impact**: What percentage of operations can benefit from FP16? How much speedup can you realistically expect? What are the implementation risks?
> 
> 4. **Production Readiness Assessment**: Which KPI targets can be met with hardware acceleration alone?

In [ ]:
# Compute projected impact vs UdaciMed targets
import numpy as np

# Baseline metrics from profiling
baseline_latency_ms = float(timing_results.get('single_sample_ms', timing_results.get('mean_ms', 0)))
baseline_throughput = float(timing_results.get('batch_throughput_samples_per_sec', timing_results.get('throughput_samples_per_sec', 0)))
param_mb = float(model_info.get('model_size_mb', 0)) if 'model_info' in globals() else 0.0

# Pick primary techniques for the plan (avoid double-counting grouped and depthwise; choose depthwise)
interp = arch_analysis['interpolation_removal']['estimated_speedup']
sep = arch_analysis['depthwise_separable']['estimated_speedup']
chan = arch_analysis['channel_organization']['estimated_speedup']
mp = deployment_analysis['mixed_precision']['estimated_speedup']
# Conservative runtime export factor (ONNX/TensorRT/EPs)
onnx_factor = 1.2
combined_speedup = float(interp * sep * chan * mp * onnx_factor)

proj_latency_ms = round(baseline_latency_ms / combined_speedup, 3) if baseline_latency_ms else None
proj_throughput = round(baseline_throughput * combined_speedup, 1) if baseline_throughput else None

# Parameter memory projection
sep_param_reduc = arch_analysis['depthwise_separable']['avg_param_reduction_percent'] / 100.0
new_param_mb = param_mb * (1 - sep_param_reduc)
new_param_mb_fp16 = new_param_mb * 0.5  # FP16

print('Projected combined speedup (interp × separable × channels_last × FP16 × ONNX):', f"{combined_speedup:.2f}x")
print('Baseline latency (ms):', baseline_latency_ms, '→ Projected latency (ms):', proj_latency_ms)
print('Baseline throughput (sps):', baseline_throughput, '→ Projected throughput (sps):', proj_throughput)
print('Model params memory (MB): baseline', round(param_mb,2), '→ separable', round(new_param_mb,2), '→ +FP16', round(new_param_mb_fp16,2))



In [ ]:
# Render final optimization strategy with quantitative justification
from IPython.display import display, Markdown

interp = arch_analysis['interpolation_removal']
sep = arch_analysis['depthwise_separable']
chan = arch_analysis['channel_organization']
mp = deployment_analysis['mixed_precision']

onnx_factor = 1.2
combined_speedup = interp['estimated_speedup'] * sep['estimated_speedup'] * chan['estimated_speedup'] * mp['estimated_speedup'] * onnx_factor
baseline_latency_ms = float(timing_results.get('single_sample_ms', timing_results.get('mean_ms', 0)))
baseline_throughput = float(timing_results.get('batch_throughput_samples_per_sec', timing_results.get('throughput_samples_per_sec', 0)))
proj_latency_ms = baseline_latency_ms / combined_speedup if baseline_latency_ms else None
proj_throughput = baseline_throughput * combined_speedup if baseline_throughput else None
param_mb = float(model_info.get('model_size_mb', 0)) if 'model_info' in globals() else 0.0
new_param_mb = param_mb * (1 - sep['avg_param_reduction_percent']/100.0)
new_param_mb_fp16 = new_param_mb * 0.5

md = f"""
### Optimization Strategy and Justification (Final Report)

UdaciMed targets: <100 MB model footprint, >2,000 samples/sec throughput, >98% sensitivity, <3 ms single‑sample latency.

Prioritized plan and rationale:

1) Remove 64→224 interpolation and adopt a lighter 3×3 stem at native resolution (64–128px)
- Why: Interpolation inflates pixels by ~12.25× and dominates early-layer compute/activations.
- Evidence: Interpolation removal estimated speedup ~{interp['estimated_speedup']:.2f}×; FLOP reduction ~{interp['avg_flop_reduction_percent']:.0f}%.
- Safety: Retrain/fine‑tune at native resolution and re‑calibrate threshold to maintain ≥98% sensitivity.

2) Replace 3×3 convolutions with depthwise‑separable blocks where channel sizes permit
- Why: 3×3 convs account for most FLOPs; separable cuts k^2·in·out to k^2·in + in·out.
- Evidence: Candidates: {sep['candidate_layers']}; average param/FLOP reduction ~{sep['avg_flop_reduction_percent']:.0f}%; estimated speedup ~{sep['estimated_speedup']:.2f}×.
- Memory: Params drop from ~{param_mb:.1f} MB → ~{new_param_mb:.1f} MB; with FP16 → ~{new_param_mb_fp16:.1f} MB.

3) Enable channels_last + in‑place activations
- Why: Better memory locality; small but free speedup on most backends.
- Evidence: Estimated speedup ~{chan['estimated_speedup']:.2f}×.

4) Mixed precision (FP16) on Tensor‑Core GPUs
- Why: Convs/MatMuls dominate compute.
- Evidence: Eligible ops ~{mp['mixed_precision_coverage_percent']:.1f}%; estimated speedup ~{mp['estimated_speedup']:.1f}×; memory ~50% lower.

5) Export to ONNX and run with ONNX Runtime/TensorRT EPs
- Why: Kernel fusion and optimized kernels; conservative extra ~1.2× speedup.

6) Batch sizing by scenario
- Real‑time: use batch=1–8 (choose min per‑sample latency);
- Throughput: increase to 16–128 until VRAM plateau; use FP16.

Projected impact from this run:
- Combined speedup ≈ {combined_speedup:.2f}× → latency ≈ {proj_latency_ms:.2f} ms (target <3 ms) and throughput ≈ {proj_throughput:.0f} sps (will scale further on GPU and larger batches).
- Model memory: params ≈ {new_param_mb:.1f} MB (separable) → ≈ {new_param_mb_fp16:.1f} MB (FP16) — comfortably <100 MB.

Implementation sequence:
A) Eliminate interpolation + lighter stem → B) Depthwise‑separable conversion of 3×3 blocks → C) channels_last + in‑place → D) FP16 training/inference → E) ONNX export + TensorRT/ORT EP → F) Batch strategy + threshold re‑calibration to keep ≥98% sensitivity.

Risk/validation:
- Track AUC, sensitivity, specificity after each stage; if sensitivity <98%, raise threshold and/or restore capacity (wider 1×1 pointwise).
- Verify numerics for FP16 on edge cases; fall back to FP32 for clinically sensitive thresholds if needed.
"""

display(Markdown(md))




## **Congratulations!** 

You have completed the model baseline analysis! This foundational work will guide all subsequent optimization efforts.

### **Summary: Key findings**
Document your analysis results using this framework:

1.  **Overall performance profile**: 

- Baseline latency: 36.9 ms single-sample (p95: 50.6 ms; std: 6.9 ms).
- Baseline throughput: 40.7 samples/sec for the profiled batch (single-sample path: 27.1 sps).
- Computational load: ~58.2 GFLOPs for the profiled input; PyTorch profiler shows ~88% of time in convolution ops.
- Memory: peak process memory ~1.56 GB; parameters ~42.6 MB, input ~1.5 MB → activations dominate.
- Architecture note: inputs are 64×64 but are upsampled to 224×224 in the model forward() to match ImageNet pretraining → a 12.25× pixel increase (224^2/64^2) that propagates through early layers.

- Targets (README): <100 MB memory, >2,000 sps throughput, >98% sensitivity, <3 ms latency.
- Current baseline meets sensitivity but misses memory, throughput and latency by large margins (≈1.56 GB, 40.7 sps, 36.9 ms).
- With architectural changes (eliminate 64→224 upsampling; depthwise/Grouped convs; lighter stem) + mixed precision and deployment acceleration (ONNX/TensorRT), the gaps are addressable in later phases.

2. **Bottlenecks**: 

- Compute bottleneck: Convolutions dominate (~88% of profiled time); 3×3 convs in residual blocks at higher resolutions are primary contributors.
- Memory bottleneck: Peak ~1.56 GB is driven by activations in early layers where spatial maps are largest; parameters are only ~43 MB.
- Latency bottleneck: Single-sample path ≈37 ms; variance (p95≈50.6 ms) suggests kernel launch/CPU scheduling overhead on CPU.
- Did you notice? 64×64 inputs are interpolated to 224×224 inside forward(), inflating both compute and activation memory by ~12.25× in the stem and first blocks. Removing this is the single biggest win.

3. **Architecture optimization**:

**Checkpoint 1 – Architecture**<br>
- Candidates for grouped/depthwise separable convolutions: the 3×3 convs in BasicBlocks across layers 2–4; they account for most FLOPs while maintaining channel sizes divisible by typical groups.
- First 7×7 stem conv at 224×224 is expensive; replacing with 3×3 stack (or a stride-2 3×3) reduces FLOPs and improves cache locality.
- Linear classifier (512→2) is negligible for both parameters and compute – not a bottleneck.
- Channel pruning/low‑rank factorization can target late-stage 3×3 convs with high redundancy while guarding sensitivity.
_- Top 2 architectural techniques with highest impact potential_<br>
_- Implementation difficulty vs expected benefit analysis_<br>
_- Estimated parameter reduction and optimization goals projections_<br>
_- Other techniques you may consider beyond those listed>>_

4. **Hardware deployment optimization**: 

**Checkpoint 2 – Deployment**<br>
- Mixed precision (FP16) is highly applicable: workload is dominated by conv/GEMM; expect ~1.5–2.0× speedup and ~50% activation/parameter memory reduction on Tensor Core GPUs.
- Batch‑size strategy: for real‑time, use batch=1 (focus on median and p95); for throughput, increase to 8–32 until latency/VRAM plateau.
- Runtime: export to ONNX and run with ONNX Runtime/TensorRT EP to fuse ops and maximize GPU utilization.
_- Mixed precision acceleration potential and implementation plan_<br>
_- Optimal batch configurations for different use cases_>>

### **Recommended optimization roadmap**

Based on the analysis, prioritize the optimization techniques and highlight the estimated combined impact on optimization goals for each phase:

**Phase 1 (Quick Wins):**

- Remove 64→224 interpolation; train/finetune at native 64–128 input with an adjusted stem (3×3).
- Replace heavy 3×3 convs with depthwise‑separable (depthwise + 1×1 pointwise) or grouped convs where channels permit.
- Consider inverted residual blocks (MobileNetV2‑style) in high‑resolution stages.

- Enable FP16 inference; calibrate threshold to preserve >98% sensitivity.
- Export to ONNX and accelerate with TensorRT/ONNX Runtime EP; use dynamic batch for screening pipelines.

- Order‑of‑magnitude impact estimate (on T4‑class GPU):
  • Remove upsampling: up to ~12× less compute/activations in early blocks.
  • Depthwise/grouped convs on 3×3 layers: ~3–8× layer‑wise FLOP reduction.
  • FP16 + TensorRT: additional ~1.5–2.5×. Combined, a >20× speed/throughput gain is realistic while cutting memory to <100 MB.

**Phase 2 (Extra Impact):**

- Quantization‑aware training to INT8 (post‑validation) for edge targets.
- Fuse BatchNorm into conv at export; prefer static shapes for better kernel selection.
- Profile multiple batch sizes to locate the knee point for your hardware.

- Deployment‑side optimizations above typically add another 1.2–2.0× beyond architecture changes and enable scaling to high‑volume screening workloads.

---

**You are now ready to move to Notebook 2: Architecture Optimization!**